# **Knowledge Graph Visualization of GraphRAG Model**

### Author: Bob Zhang
### NetID: zz347
### Department: MIDS, Duke University

## **Introduction**

The demand for transparency and trust in decision-making processes has become
paramount. This is particularly critical in high-stakes domains such as **healthcare** and
**critical care**, where AI-driven decisions directly impact patient outcomes. Recent AI
models such as **GraphRAG** integrate powerful neural network architectures like
transformers with graph-based retrieval and reasoning mechanisms. But the lack of
inherent explainability raises concerns about how physicians and healthcare
professionals can interpret its decisions.

> This project aims to bridge this gap by enhancing the explainability of GraphRAG through the implementation of a visualization method that allows users to explore the underlying knowledge graph. By enabling interaction with the graph's entities and relationships, this approach provides insight into the model's retrieval mechanisms, fostering greater transparency and trust in its decision-making process.

## **Methodology**

This project demonstrates the construction of GraphRAG from scratch, integrating a graph database powered by **Neo4j** and comparing model-generated answers with ground truth data. The methodology involves building the knowledge graph, implementing the retrieval mechanism, and evaluating the model's performance. The process utilizes **LangChain**, **LLM models (Llama 3.1 and GPT-4 Turbo)**, and **OpenAI’s text-embedding-3-large** for entity extraction and retrieval.

---

### **1. Building GraphRAG and Database Integration**

- **Knowledge Graph Construction**:
  - The knowledge graph is constructed using the **LangChain framework**, which facilitates the integration of large language models (LLMs) for entity and relationship extraction.
  - **LLM Models**:
    - **Llama 3.1** and **GPT-4 Turbo** are employed to extract entities and relationships from input text, enabling the creation of a robust graph structure.

- **Database Configuration**:
  - The extracted entities and relationships are stored in a **Neo4j graph database**.
  - Neo4j can be deployed:
    - **Using Docker**: Recommended for users who do not have Neo4j pre-installed on their desktop.
    - **On Desktop**: For users with a local Neo4j setup.
  - The graph database utilizes **Cypher query language**, similar to SQL in relational databases, to manage and retrieve data.

---

### **2. Enhancing Retrieval with Vector Indexing**

- **Vectorization of Entities and Queries**:
  - OpenAI’s **text-embedding-3-large** model is used to vectorize:
    - Input queries.
    - Extracted entities from the knowledge graph.
  - This enables the construction of a **vector index** to enhance retrieval efficiency.

- **Hybrid Retrieval Mechanism**:
  - Combines graph-based retrieval using Cypher queries with vector similarity search to provide accurate and contextually relevant results.

---

### **3. Workflow Implementation**

- **LangChain Workflow**:
  - LangChain orchestrates the entity extraction, knowledge graph construction, and retrieval mechanism, seamlessly integrating LLMs and the Neo4j database.

- **Model Querying and Evaluation**:
  - Input queries are processed through GraphRAG, and the model retrieves information using the hybrid mechanism.
  - The model-generated answers are compared to ground truth data to evaluate accuracy and reliability.

---

### **4. Tools and Frameworks**

- **LangChain**: Provides the framework for knowledge graph creation and integration with LLMs.
- **LLMs**:
  - **Llama 3.1** and **GPT-4 Turbo** for entity and relationship extraction.
- **Neo4j**: Graph database used for storing and querying structured relationships.
  - Deployed via Docker container for accessibility.
- **OpenAI text-embedding-3-large**: Used for vectorizing entities and input queries to build the vector index.

---

### **5. Neo4j Setup and Queries**

- **Deployment Options**:
  - **Docker Container**: Recommended due to ease of setup and portability.
  - **Desktop**: For advanced users with Neo4j pre-installed.
- **Cypher Query Language**:
  - The inherent query language of Neo4j, used to interact with the graph database and retrieve nodes and relationships efficiently.


### **Enviroment Setup**

To ensure the project environment was properly configured, I installed all the required Python modules and packages necessary for implementing and testing the solution. The setup involved the use of several libraries from LangChain, Neo4j, and related ecosystems, along with utilities for handling embeddings, text processing, and graph-based operations. After installing these packages using pip, I verified their functionality by testing basic operations, such as connecting to the Neo4j database, loading documents, and running queries. This environment setup provides the foundational tools required for implementing the project's graph-based retrieval and reasoning workflows.

In [12]:
# %pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

In [1]:
from langchain_core.runnables import (
    RunnablePassthrough,
)  # For creating and managing runnable tasks
from langchain_core.prompts import (
    ChatPromptTemplate,
)  # For managing chat-based prompt templates
from pydantic import BaseModel, Field
from langchain_core.output_parsers import (
    StrOutputParser,
)  # For parsing model outputs into structured formats
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain.text_splitter import TokenTextSplitter
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars


# from langchain_ollama import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

# Force reload of the .env file
load_dotenv(override=True)

True

### Initiate Neo4j Graph

To enable seamless integration between the Neo4j graph database and the project, the graph connection was initialized using the Neo4jGraph class from the LangChain community module. The following code was used to establish the connection:

In [2]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

/var/folders/q4/0d5y6tc142n0r8342v4l5tj40000gn/T/ipykernel_11456/2338067899.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


Once the Neo4jGraph object is initialized, it creates a connection to the Neo4j database, allowing subsequent operations to interact with the graph.The graph object serves as the entry point for all interactions with the Neo4j database. It facilitates: 
- Running Cypher queries to retrieve or manipulate nodes and relationships.

- Building and visualizing the knowledge graph by integrating it with other components of the LangChain framework.

- Storing or retrieving entities and their relationships to enhance the explainability of GraphRAG's decision-making process.

# Documents indexation

In [9]:
loader = TextLoader(file_path="dataset/ROSE.txt")
docs = loader.load()

text_splitter = TokenTextSplitter(
    chunk_size=250, chunk_overlap=24
)  # change chunk size and overlap to your needs
documents = text_splitter.split_documents(documents=docs)
print(len(documents))
# print the type of documents
print(type(documents))
print(type(documents[0]))

26
<class 'list'>
<class 'langchain_core.documents.base.Document'>


# Define LLM and Convert Docs to Graph using LLM Graph Transformer

In [10]:
llm_type = os.getenv("LLM_TYPE", "not ollama")
if llm_type == "ollama":
    llm = ChatOllama(model="llama3.1", temperature=0, format="json")
else:
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

llm_transformer = LLMGraphTransformer(llm=llm)
llm_transformer
graph_documents = llm_transformer.convert_to_graph_documents(documents)

AttributeError: module 'openai' has no attribute 'OpenAI'

In [22]:
# graph_documents

[GraphDocument(nodes=[Node(id='Acute Respiratory Distress Syndrome', type='Condition', properties={}), Node(id='Mechanical Ventilation', type='Treatment', properties={}), Node(id='Continuous Neuromuscular Blockade', type='Treatment', properties={}), Node(id='Cisatracurium', type='Substance', properties={}), Node(id='Deep Sedation', type='Treatment', properties={}), Node(id='Usual-Care Approach', type='Treatment', properties={}), Node(id='Lighter Sedation Targets', type='Treatment', properties={}), Node(id='High Peep', type='Treatment', properties={}), Node(id='In-Hospital Death', type='Outcome', properties={})], relationships=[Relationship(source=Node(id='Acute Respiratory Distress Syndrome', type='Condition', properties={}), target=Node(id='Mechanical Ventilation', type='Treatment', properties={}), type='TREATED_WITH', properties={}), Relationship(source=Node(id='Continuous Neuromuscular Blockade', type='Treatment', properties={}), target=Node(id='Cisatracurium', type='Substance', pro

## Storing Graph Documents into Neo4j

In [23]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True, include_source=True)

# Visualization of Graph Database

In [5]:
def showGraph():
    driver = GraphDatabase.driver(
        uri=os.getenv("NEO4J_URL"),
        auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD")),
    )
    session = driver.session()
    widget = GraphWidget(graph=session.run("MATCH (s)-[r]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = "id"
    return widget


showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

# Define Vector Index as Retriever

In [3]:
vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    search_type="hybrid", # combining vector index and keyword index search for better results
    node_label='Document',#"Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
)
vector_retriever = vector_index.as_retriever()

ClientError: {code: Neo.ClientError.Schema.IndexWithNameAlreadyExists} {message: There already exists an index called 'keyword'.}

In [ ]:
driver = GraphDatabase.driver(
    uri=os.getenv("NEO4J_URL"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD")),
)


def create_fulltext_index(tx):
    query = """
    CREATE FULLTEXT INDEX `fulltext_entity_id`
    FOR (n:__Entity__)
    ON EACH [n.id];
    """
    tx.run(query)


# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")


# Call the function to create the index
try:
    create_index()
except:
    # print("Error creating the fulltext index.")
    pass

# Close the driver connection
driver.close()

In [ ]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the medical conditions, concept, treatment or outcomes entities that "
        "appear in the text",
    )


parser = PydanticOutputParser(pydantic_object=Entities)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are extracting medical conditions entities from the text.
        Return the entities as a JSON object with a 'names' field containing an array of strings.
        
        The output should be formatted as a JSON object with the following structure:
        {{"names": ["entity1", "entity2", ...]}}
        
        DO NOT include any other text or explanation.""",
        ),
        ("human", "Extract entities from this text: {question}"),
    ]
)

inf_llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.5)

entity_chain = prompt | inf_llm | parser

response = entity_chain.invoke({"question": "What is ards?"})
print(response.names)

['ards']


In [ ]:
entity_chain.invoke("What is ARDS and Sepsis?")

Entities(names=['ARDS', 'Sepsis'])

In [ ]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """

    result = ""
    entities = entity_chain.invoke(question)
    print(entities.names)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r]->(neighbor)
              RETURN node.id+ ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        print(f"response: {response}")
        result += "\n".join([el["output"] for el in response])
    return result

In [ ]:
graph_output = graph_retriever("What is ARDS?")
print(graph_output)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 116} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r]->(neighbor)\n              RETURN node.id+ ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


['ARDS']
response: [{'output': 'Ards - REQUIRES -> Mechanical Ventilation'}, {'output': '9c5ed7b2bec8619b3b52c82c472b0401 - MENTIONS -> Ards'}, {'output': 'abbbdb63e743f199c244e0090c6cd550 - MENTIONS -> Ards'}, {'output': 'cf1437c35ff826ae08c902089c03d00d - MENTIONS -> Ards'}, {'output': '2831ca76007d23bcc0d4d000ff3ba724 - MENTIONS -> Ards'}, {'output': '86ac7f7e70073e0e54b9732aaf3eb542 - MENTIONS -> Ards'}, {'output': 'f468d6266a93d15be992871df61407e5 - MENTIONS -> Ards'}, {'output': '0caa74b372dfc97f26a18da2df094740 - MENTIONS -> Ards'}, {'output': 'fa8c52ff75b0742806a26b4350ed0599 - MENTIONS -> Ards'}, {'output': 'a76abbabe70f339458a6f33fc3ea38e7 - MENTIONS -> Ards'}, {'output': '1e8fdeafc8c166452355dad958b0e62b - MENTIONS -> Ards'}, {'output': 'e395f01be42133b95ecd6fe0667331a4 - MENTIONS -> Ards'}, {'output': 'Peep - TREATMENT_FOR -> Ards'}, {'output': 'Neuromuscular_Blockade - TREATMENT_FOR -> Ards'}, {'output': 'Low Tidal Volume Ventilation - TREATMENT_FOR -> Ards'}, {'output': '

In [ ]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": full_retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke(
    input="In patients with moderate to severe ARDS, "
    "does the early use of continuous neuromuscular blockade with cisatracurium improve mortality when used with current light sedation protocols?"
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 116} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r]->(neighbor)\n              RETURN node.id+ ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


['moderate to severe ARDS']
response: [{'output': 'Ards - REQUIRES -> Mechanical Ventilation'}, {'output': '6ccf6204fc494db33f778fafb78ffe5e - MENTIONS -> Moderate-To-Severe Ards'}, {'output': '3b8e2f9d215723aa6531a4ad490371a4 - MENTIONS -> Moderate-To-Severe Ards'}, {'output': 'e395f01be42133b95ecd6fe0667331a4 - MENTIONS -> Moderate-To-Severe Ards'}, {'output': 'Rose Trial - INVOLVES -> Moderate-To-Severe Ards'}, {'output': 'Higher Peep Strategy - USED_FOR -> Moderate-To-Severe Ards'}, {'output': 'Early And Continuous Infusion Of Cisatracurium - USED_FOR -> Moderate-To-Severe Ards'}, {'output': '9c5ed7b2bec8619b3b52c82c472b0401 - MENTIONS -> Ards'}, {'output': 'abbbdb63e743f199c244e0090c6cd550 - MENTIONS -> Ards'}, {'output': 'cf1437c35ff826ae08c902089c03d00d - MENTIONS -> Ards'}, {'output': '2831ca76007d23bcc0d4d000ff3ba724 - MENTIONS -> Ards'}, {'output': '86ac7f7e70073e0e54b9732aaf3eb542 - MENTIONS -> Ards'}, {'output': 'f468d6266a93d15be992871df61407e5 - MENTIONS -> Ards'}, {'outp

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'No, the early use of continuous neuromuscular blockade with cisatracurium does not improve mortality in patients with moderate to severe ARDS when used with current light sedation protocols. The trial was stopped for futility, indicating that this approach did not result in significantly lower mortality at 90 days compared to usual care with lighter sedation targets.'